# Download the demo data for a given instrument

In [ ]:
import utilities
import os
import requests
import tarfile
from urllib.parse import urlparse
from tqdm import tqdm

## Input the instrument name

In [ ]:
selector = utilities.TextSelector(default_description='Instrument name:', default_input='uves')
selector.display()

In [ ]:
instrument_name = selector.get_input().lower()

extractor = utilities.URLExtractor('https://www.eso.org/sci/software/pipe_aem_table.html', instrument_name)
link = extractor.extract_urls()
print(f"The Demo Data link for {instrument_name.upper()} is: {link}")

## Locate and download the demo data tar.gz and unpack it

In [ ]:
# Extract the filename from the URL
filename = os.path.basename(urlparse(link).path)

def safe_extract_filter(member, tar):
    # Prevent path traversal attacks
    target_path = os.path.abspath(os.path.join('.', member.name))
    if not target_path.startswith(os.path.abspath('.')):
        raise Exception("Attempted Path Traversal in Tar File")
    return member

# Download the file with a progress indicator
response = requests.get(link, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte

with open(filename, 'wb') as f, tqdm(total=total_size, unit='iB', unit_scale=True) as progress_bar:
    for chunk in response.iter_content(chunk_size=block_size):
        f.write(chunk)
        progress_bar.update(len(chunk))

with tarfile.open(filename, 'r:gz') as tar:
    tar.extractall(path='.', filter=safe_extract_filter)

os.remove(filename)